# **CSU Machine Learning Probabilities**
<a href="https://githubtocolab.com/csteele2/Wx4Colab/blob/master/CSU_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a> <br/>

A notebook to look at future or past [Colorado State Machine Learning Probabilities](http://schumacher.atmos.colostate.edu/hilla/csu_mlp/index.php)  <s>Excessive Rainfall or</s> Severe Weather Outlooks. <s>There are a few different versions you can look at. The EROs have multiple versions that vary based on the training dataset and/or input models used.</s> For severe, day 1 - 3 probs are by element, for days 3+, it's aggregate only. <s>While plotting ERO, for verification purposes, you can overlay LSRs (using the Iowa State API), and/or WPC verification layers, such as "practically perfect" analysis, as well as UFVS Stage IV QPE obs. For severe verification, only LSRs are available. </s><br />
 It should be as simple as running the first cell to setup the environment, then setting options in the form and running that cell. Subsequent plots during the session can be made by making changes to the form and reunning that cell, no need to keep trying to install and import packages with the first cell.

 > 5-Aug-2024: Reset time! A new data source and new visualization has (hopefully temporarily) reduced functionality. The bad news - the only thing display now is the latest run convective severe probabilities. The good news - it's on an interactive map! ERO, support for archive and LSRs, etc. will return in time.

 > 10-Oct-2023: Since cfgrib kinda stopped working with the existing implementation of condacolab, and since condacolab isn't necessary anymore now that google colab is on py 3.10+, removed the install of conda colab (which removed a step and error message! yay!). Also synced some changes with CONUS projection and titles.

*29-December-2022 <br/>
Caleb Steele* <br/>
https://github.com/csteele2/Wx4Colab



## **1 - Install and import**

This will install and import everything we need. You need only run this once per session, then you can make all the changes to the form and make as many plots as you wish without having to rerun this cell (unless your session expires and spins down).

In [ ]:
!pip install ecmwflibs
!pip install eccodes
!pip install cfgrib
!pip install cartopy xarray geoviews

import numpy as np
import os
from urllib.request import urlretrieve

import matplotlib
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
import matplotlib.axes as maxes
import matplotlib.patheffects as PathEffects
from matplotlib.path import Path
from matplotlib.textpath import TextToPath
from matplotlib.font_manager import FontProperties
matplotlib.rcParams['font.sans-serif'] = 'Liberation Sans'
matplotlib.rcParams['font.family'] = "sans-serif"
from cartopy import cartopy, crs as ccrs, feature as cfeature

import xarray as xr
import pandas as pd
import json
import geoviews as gv
import geoviews.feature as gf
from geoviews import opts, tile_sources as gvts
from bokeh.models import Title

from datetime import datetime, timedelta, timezone

import warnings
warnings.filterwarnings("ignore")

## **2 - Set options and go!**

Set all your options in the form below and go!

In [ ]:
#################### FORM STUFF HERE ###################################################################################
# Removed = date/cycle selection

current_time = datetime.now(timezone.utc)

if current_time.hour < 5:
  init_date = current_time.replace(hour=12, minute=0, second=0, microsecond=0) - timedelta(days=1)
  init_cycle = 12
elif current_time.hour < 17:
  init_date = current_time.replace(hour=0, minute=0, second=0, microsecond=0)
  init_cycle = 0
else:
  init_date = current_time.replace(hour=12, minute=0, second=0, microsecond=0)
  init_cycle = 12


#@markdown <br /> <h2> <b> Which Day? </b></h2>
day = 3 #@param {type:"raw"} [1, 2, 3, 4, 5, 6, 7, 8]
day = int(day)

# Removed - ERO/Severe Selection
hazard = "Severe"
#@markdown <b> If Day < 3 severe, which hazard? </b>
#@markdown <br />[Hazard will only be "severe" (combined) beyond day 3]
severe_hazard = "SEVERE" #@param ["TOR", "HAIL", "WIND", "SEVERE"]

#@markdown  <br /> <h2> <b> Select map theme </b></h2>
map_theme = "ESRI Terrain" # @param ["ESRI Imagery","ESRI Terrain","ESRI Light Grey"]


dayHourDict = {
    0: {
      1 : 36,
      2: 60,
      3: 84,
      4: 108,
      5: 132,
      6: 156,
      7: 180,
      8: 204
    },
    12: {
      2: 48,
      3: 72,
      4: 96,
      5: 120,
      6: 144,
      7: 168,
      8: 192
    }

}
valid_date = init_date + timedelta(hours = dayHourDict[init_cycle][day])
valid_date_start = valid_date - timedelta(hours = 24)

if dayHourDict[init_cycle][day] > 84:
  severe_hazard = "severe"
  print("   > Only general severe available at this range")

title_text = f"CSU MLP · Day {str(day)} {severe_hazard.capitalize()}\nInit: {init_date.strftime('%HZ %d-%b-%Y')} | Valid: {valid_date.strftime('%HZ %m-%d-%Y')}"

# set up some global variables here

latloncrs = ccrs.PlateCarree()
WebMerc = ccrs.Mercator.GOOGLE


# Set up the URL
url_base = f"https://nomads.ncep.noaa.gov/pub/data/nccf/com/spc_post/prod/spc_post.{init_date.strftime('%Y%m%d')}/gefs_mlp/"
file = f"spc_post.t{init_date.strftime('%Hz')}.gefsmlp_{severe_hazard.lower()}_std.f{str(dayHourDict[init_cycle][day]).zfill(3)}.grib2"
url = url_base+file


################## DOWNLOAD HERE #######################################################################################
print('Looking for grib file...')
if os.path.exists(file):
  print("   > Grib already exists")
else:
  try:
    print("   > Downloading grib")
    urlretrieve(url, file)
  except:
    print(f"     >> Unable to get file [{url}]")

print('   > Processing grib file')
mlp_ds = xr.open_dataset(file, engine="cfgrib")
dkey = list(mlp_ds.keys())[0]
mlp_ds[dkey] = mlp_ds[dkey]*100



################ Make a geoviews dataset and show an interactive map ###################################################
# make a geoviews dataset
mlp_gv = gv.Dataset(mlp_ds, ['longitude', 'latitude',], dkey, crs=latloncrs)
# convert to filled contours
contours = mlp_gv.to(gv.FilledContours)
# set up the contour colors
SVR_COLS = ["#ffffff00", "#dcdcdcaa", "#b9c9baaa", "#f1c86aaa", "#ed7645aa", "#ce2754aa", "#a20435aa","#630157aa"]
SVR_LEVS = [0.0, 2, 5, 10, 15, 30, 45, 60, 100]
#SVR_CMAP, SVR_NORM = matplotlib.colors.from_levels_and_colors(SVR_LEVS, SVR_COLS, extend='max')

# set the initial map bounds
xymin = WebMerc.transform_point(-130, 25, latloncrs)
xymax = WebMerc.transform_point(-64, 50, latloncrs)

print('Making interactive map...')

# use the bokeh back end
gv.extension('bokeh')

# setup the map theme (chosen from the form)
match map_theme:
  case "ESRI Imagery":
    map_layers = gvts.EsriImagery * gvts.EsriReference(level='overlay')
  case "ESRI Terrain":
    map_layers = gvts.EsriTerrain * gvts.EsriReference(level='overlay')
  case "ESRI Light Grey":
    map_layers = gvts.EsriWorldLightGrayBase * gvts.EsriWorldLightGrayReference(level='overlay')
  case _:
    map_layers = gvts.EsriTerrain * gvts.EsriReference(level='overlay')

# now just put everything together and show it
map =  map_layers * contours.opts(cmap=SVR_COLS, color_levels=SVR_LEVS, colorbar=True, width=1000, height=650, line_color=None, title=title_text)
map.opts(projection=WebMerc, xlim=(xymin[0], xymax[0]), ylim=(xymin[1], xymax[1]), global_extent=False)